In [7]:
import pandas as pd
import numpy as np

In [8]:
#columns = ['current_state', 'action', 'next_state', 'probability']
#df = pd.DataFrame(pd.read_csv('mdp_1d_statedata.xls', header = None),columns=columns)
df = pd.read_csv('mdp_2d_data.xls')
type(df)

pandas.core.frame.DataFrame

In [9]:
df.head()

,Inventory,Price,probability,nx_Inventory,nx_Price,action
0,10,5,0.6,9,5,a
1,10,5,0.4,8,5,a
2,10,5,0.3,9,6,a


In [10]:
'''
columns = ['current_state', 'action', 'next_state', 'probability']
df.columns = columns
'''

"\ncolumns = ['current_state', 'action', 'next_state', 'probability']\ndf.columns = columns\n"

In [11]:
df.head()

,Inventory,Price,probability,nx_Inventory,nx_Price,action
0,10,5,0.6,9,5,a
1,10,5,0.4,8,5,a
2,10,5,0.3,9,6,a


In [12]:
nx_state_var = [col for col in df.columns if col.startswith('nx') and col != 'action' and col != 'probability' and col != 'reward']
#nx_state_var = ['nx_capacity']
state_var = [col.replace('nx_', '') for col in nx_state_var]

In [28]:
df['current_state'] = df.loc[:, state_var].apply(lambda row: ', '.join(map(str, row)), axis=1)
df['next_state'] = df.loc[:, nx_state_var].apply(lambda row: ', '.join(map(str, row)), axis=1)

In [29]:
states = df['current_state'].unique()
print(states)
print(type(states))

['10, 5']
<class 'numpy.ndarray'>


In [30]:
random_samples = []
num_samples = 100000  # Number of samples to generate

for _ in range(num_samples):
    random_state = np.random.choice(states)
    #print(random_state)
    sample_action = np.random.choice((df[df['current_state'] == random_state]).action.unique())
    #print(sample_action)
    sample_row = df[(df['current_state'] == random_state) & (df['action'] == sample_action)].sample(n=1, weights='probability').iloc[0]

    # Extract the values from the selected row
    current_state = sample_row['current_state']
    action = sample_row['action']
    next_state = sample_row['next_state']

    # Add the sample to the list
    random_samples.append([current_state, action, next_state])
    #print(random_samples)

In [31]:
df_samples = pd.DataFrame(random_samples, columns=['current_state', 'action', 'next_state'])
df_samples.head()

,current_state,action,next_state
0,"10, 5",a,"9, 6"
1,"10, 5",a,"8, 5"
2,"10, 5",a,"8, 5"
3,"10, 5",a,"9, 6"
4,"10, 5",a,"9, 5"


In [32]:
df_empirical_probs = df_samples.groupby(['current_state', 'action', 'next_state']).size().reset_index(name='count')
print(df_empirical_probs)
df_empirical_count = df_samples.groupby(['current_state', 'action']).size().reset_index(name='count_total')
print(df_empirical_count)

df_empirical_probs = df_empirical_probs.merge(df_empirical_count, on=['current_state', 'action'])
df_empirical_probs['empirical_probability'] = df_empirical_probs['count'] / df_empirical_probs['count_total']
print(df_empirical_probs)


  current_state action next_state  count
0         10, 5      a       8, 5  30818
1         10, 5      a       9, 5  46368
2         10, 5      a       9, 6  22814
  current_state action  count_total
0         10, 5      a       100000
  current_state action next_state  count  count_total  empirical_probability
0         10, 5      a       8, 5  30818       100000                0.30818
1         10, 5      a       9, 5  46368       100000                0.46368
2         10, 5      a       9, 6  22814       100000                0.22814


In [33]:
df_merged = df.merge(df_empirical_probs, on=['current_state', 'action', 'next_state'], how='left')
# Check if the empirical probabilities match the specified probabilities
df_merged['prob_diff'] = df_merged['probability'] - df_merged['empirical_probability']
all_probs_match = np.allclose(df_merged['probability'], df_merged['empirical_probability'])

# Display the comparison results
print(df_merged)
print("\nDo all probabilities match?", all_probs_match)

   Inventory  Price  probability  nx_Inventory  nx_Price action current_state  \
0         10      5          0.6             9         5      a         10, 5   
1         10      5          0.4             8         5      a         10, 5   
2         10      5          0.3             9         6      a         10, 5   

  next_state  count  count_total  empirical_probability  prob_diff  
0       9, 5  46368       100000                0.46368    0.13632  
1       8, 5  30818       100000                0.30818    0.09182  
2       9, 6  22814       100000                0.22814    0.07186  

Do all probabilities match? False
